In [ ]:
# Search Application 

import tkinter as tk
from tkinter import ttk, messagebox
from elasticsearch import Elasticsearch
import psycopg2
import time 

input_fields  = {}


# Connection to Elasticsearch
es = Elasticsearch("http://localhost:9200")

# PostgreSQL connection parameters
db_params = {
    'dbname': 'postgres',
    'user': 'postgres',
    'password': 'YourPassword',
    'host': 'localhost',
    'port': '5432'
}

# Connect to PostgreSQL
conn = psycopg2.connect(**db_params)
cur = conn.cursor()

# Cache Set Up
cache = []
cur.execute(f"SELECT * FROM user_meta_data ORDER BY followers DESC")
results = cur.fetchall()
top10_users = results[0:9]
for users in top10_users:
    cur.execute(f"SELECT * FROM original_tweet_meta_data WHERE user_id = {users[0]} limit 10")
    results = cur.fetchall()
    for result in results:
        cache.append((result,10000000000000))
top_hashtags = ['Corona','Mattarella','25Aprile','corona','Covid_19','AltaredellaPatria',
                'COVID19','Liberazione','coronavirus','Coronavirus','China','cumartesi',
                'Covid19','Iftar','CoronaVirus','lockdown','COVID','FightCoronaNotActivists',
                'BREAKING','StayAtHome','eAgendaAajTak','2020WasFunUntil','TamilNadu','Master','covid19',
                'USA','Politics','respect','Lockdown','Economy','TN','India','level4lockdown','Trump',
                'Weltpolitik','StayHome''SunNews','IndiaFightsCorona','Coronakrise','25aprile','FechadoComBolsonaro',
                'BISP','BankersUnited','CentralVista','covid','coronavirusdeutschland','AkshayKumar','RMM','stayhome'
                ,'Virus','COVID2019']
for hashtag in top_hashtags:
    cur.execute(f"SELECT * FROM original_tweet_meta_data WHERE hashtags LIKE %s limit 10", ('%' + hashtag + '%',))
    results = cur.fetchall()
    for result in results:
        if (result,10000000000000) in cache:
            continue
        else:
            cache.append((result,10000000000000))


'''Search Functions'''

def search_by_user_id(user_input):
    try:
        global cache
        # Initialize a set to store unique tweet IDs
        unique_tweet_ids = set()
        results = []
        
        curr_time = time.time()
        for i in range(len(cache)):
            if str(cache[i][0][1]) == user_input:
                unique_tweet_ids.add(cache[i][0][0])
                results.append(cache[i][0])
                if cache[i][1] != 10000000000000:
                    cache[i] = (cache[i][0],curr_time)
        
        # Search in Elasticsearch
        query = {
            "query": {
                "term": {"user_id": user_input}
            }
        }
        response = es.search(index="twitter-index-final", body=query)

        additions = 0
        # Iterate over Elasticsearch hits
        for hit in response['hits']['hits']:
            table_name = hit['_source']['table_name']
            tweet_id = hit['_source']['tweet_id']
            
            # Fetch tweet data only if it hasn't been fetched yet
            if tweet_id not in unique_tweet_ids:
                print(tweet_id)
                cur.execute(f"SELECT * FROM {table_name} WHERE tweet_id = %s", (tweet_id,))
                unique_tweet_ids.add(tweet_id)
                results.extend(cur.fetchall())
                if additions < 50:
                    if len(cache) == 1000:

                        cache = sorted(cache, key=lambda x:x[1],reverse=True)
                        cache[999] = (results[len(results)-1],curr_time)
                        additions += 1
                    else:
                        cache.append((results[len(results)-1],curr_time))
                        additions += 1
                
        
        return results

    except Exception as e:
        print("An error occurred:", str(e))
        return []


def search_by_twitter_id(user_input):
    # Check Cache 
    try:
        global cache
        unique_tweet_ids = set()
        results = []
        
        curr_time = time.time()
        for i in range(len(cache)):
            if str(cache[i][0][0]) == user_input:
                unique_tweet_ids.add(cache[i][0][0])
                results.append(cache[i][0])
                if cache[i][1] != 10000000000000:
                    cache[i] = (cache[i][0],curr_time)
        # Check Elastic Search Index
        query = {
            "query": {
                "term": {"tweet_id": user_input}
            }
        }
        response = es.search(index="twitter-index-final", body=query)
        
        additions = 0
        # Pull data from SQL
        for hit in response['hits']['hits']:
            table_name = hit['_source']['table_name']
            tweet_id = hit['_source']['tweet_id']
            if tweet_id not in unique_tweet_ids:
                cur.execute(f"SELECT * FROM {table_name} WHERE tweet_id = %s", (tweet_id,))
                unique_tweet_ids.add(tweet_id)
                results.extend(cur.fetchall())
                if additions < 50:
                    if len(cache) == 1000:

                        cache = sorted(cache, key=lambda x:x[1],reverse=True)
                        cache[999] = (results[len(results)-1],curr_time)
                        additions += 1
                    else:
                        cache.append((results[len(results)-1],curr_time))
                        additions += 1
                
        return results

    except Exception as e:
        print("An error occurred:", str(e))
        return []


def search_by_time_frame(from_time, to_time):
    # Pull data from SQL directly 
    cur.execute("""
        SELECT tweet_id, user_name, tweet_text, created_at, retweet_count, favorite_count, user_followers
        FROM tweets_project
        WHERE created_at BETWEEN %s AND %s
        ORDER BY (retweet_count + favorite_count + user_followers) DESC
    """, (from_time, to_time))
    return cur.fetchall()
    
   # return results


def search_by_hashtags(user_input):
    # Check Cache
    try:
        global cache
        unique_tweet_ids = set()
        results = []
        
        curr_time = time.time()
        hashtag == '"'+user_input+'"' 
        for i in range(len(cache)):
            if hashtag in cache[i][0][4]:
                unique_tweet_ids.add(cache[i][0][0])
                results.append(cache[i][0])
                if cache[i][1] != 10000000000000:
                    cache[i] = (cache[i][0],curr_time)

        
        # Check Elastic Search Index
        query = {
            "query": {
                "match": {"hashtags": user_input}
            }
        }
        response = es.search(index="twitter-index-final", body=query)
        
        additions = 0
        
        # Pull data from SQL
        for hit in response['hits']['hits']:
            table_name = hit['_source']['table_name']
            tweet_id = hit['_source']['tweet_id']
            if tweet_id not in unique_tweet_ids:
                cur.execute(f"SELECT * FROM {table_name} WHERE hashtags LIKE %s", ('%' + user_input + '%',))
                unique_tweet_ids.add(tweet_id)
                results.extend(cur.fetchall())
                
                if additions < 50:
                    if len(cache) == 1000:
                        cache = sorted(cache, key=lambda x:x[1],reverse=True)
                        cache[999] = (results[len(results)-1],curr_time)
                        additions += 1
                    else:
                        cache.append((results[len(results)-1],curr_time))
                        additions += 1
                
        return results

    except Exception as e:
        print("An error occurred:", str(e))
        return []


def search_by_tweet_text(user_input):
    # Define the Elasticsearch query body
    search_body = {
        "query": {
            "match": {
                "tweet_text": user_input
            }
        },
        "size": 10  # Adjust size as needed to control how many results to fetch
    }
    
    # Execute search on Elasticsearch
    es_results = es.search(index="tweets_final", body=search_body)
    
    # Extract tweet IDs from the search results
    tweet_ids = [hit['_id'] for hit in es_results['hits']['hits']]
    
    # Check if there are any IDs to fetch details for
    if tweet_ids:
        # Convert tweet_ids to integers if necessary
        tweet_ids = [int(tweet_id) for tweet_id in tweet_ids]
        
        # Execute SQL query to fetch tweet details from PostgreSQL
        cur.execute("""
            SELECT tweet_id, user_name, tweet_text, retweet_count, favorite_count, user_followers
            FROM tweets_project
            WHERE tweet_id = ANY(%s)
            ORDER BY (retweet_count + favorite_count + user_followers) DESC  -- Improved ranking based on sum of interactions and followers
        """, (tweet_ids,))
        
        # Fetch all results from the query
        results = cur.fetchall()
        return results
    else:
        # Return an empty list or message if no tweets found
        return "No tweets found matching the search criteria."



# This function can now be called with user input and will return the search results



# Tkinter GUI setup
root = tk.Tk()
style = ttk.Style()
center_frame = ttk.Frame(root)
frame_dropdown = ttk.Frame(center_frame)
frame_inputs = ttk.Frame(center_frame)
search_options = ['Search by User ID', 'Search by Tweet ID', 'Search by Time Frame', 'Search by Hashtag', 'Search by Tweet Text']
search_type = tk.StringVar(value=search_options[0])

# Update inputs based on search type
def update_inputs(event=None):
    global input_fields
    for widget in frame_inputs.winfo_children():
        widget.destroy()
    
    input_fields = {}  # Reinitialize the dictionary to clear old references
    search_option = search_type.get()
    if search_option == 'Search by Time Frame':
        create_input_field("Start Time:", "End Time:")
    else:
        input_label = search_option.replace('Search by ', 'Enter ')
        create_input_field(f"{input_label}:")


def create_input_field(*label_texts):
    for label_text in label_texts:
        label = ttk.Label(frame_inputs, text=label_text, font=('Arial', 16))
        label.pack(anchor='center')
        input_field = ttk.Entry(frame_inputs, font=('Arial', 16), width=30)
        input_field.pack(anchor='center', pady=5)
        input_field.focus()
        input_fields[label_text.strip(':')] = input_field

def perform_search():
    search_type_selection = search_type.get()

    # Safely collect input values from existing widgets
    input_values = [field.get() for field in input_fields.values() if field.winfo_exists()]

    start_time = time.time()  # Start timing the search operation
    # Dynamically choosing the search function based on the selected search type
    if search_type_selection == 'Search by User ID' and len(input_values) >= 1:
        results = search_by_user_id(input_values[0])
    elif search_type_selection == 'Search by Tweet ID' and len(input_values) >= 1:
        results = search_by_twitter_id(input_values[0])
    elif search_type_selection == 'Search by Time Frame' and len(input_values) >= 2:
        results = search_by_time_frame(input_values[0], input_values[1])
    elif search_type_selection == 'Search by Hashtag' and len(input_values) >= 1:
        results = search_by_hashtags(input_values[0])
    elif search_type_selection == 'Search by Tweet Text' and len(input_values) >= 1:
        results = search_by_tweet_text(input_values[0])
    else:
        results = []

    end_time = time.time()  # End timing
    messagebox.showinfo("Search Completed", f"Total search operation took {end_time - start_time:.2f} seconds")
    show_results_window(results)
    



def show_results_window(search_results):
    results_window = tk.Toplevel(root)
    results_window.title("Search Results")

    # Adjusted on_close function to call update_inputs
    def on_close():
        root.focus_set()
        update_inputs()  # Update the input fields after closing the results window
        results_window.destroy()

    results_window.protocol("WM_DELETE_WINDOW", on_close)

    listbox = tk.Listbox(results_window, width=50, height=10)
    listbox.pack(pady=20, padx=20)
    for result in search_results:
        listbox.insert(tk.END, result)
    
    # Wait for the results window to close before moving on
    root.wait_window(results_window)



def setup_gui():
    root.title("Search Application")
    root.geometry("700x500")
    center_frame.pack(expand=True)
    dropdown_label = ttk.Label(frame_dropdown, text="Select search type:", font=('Arial', 16))
    dropdown_label.pack(side='left', padx=10)
    search_menu = ttk.OptionMenu(frame_dropdown, search_type, search_options[0], *search_options, command=update_inputs)
    search_menu.pack(side='left', padx=10)
    frame_dropdown.pack(side='top', pady=10)
    frame_inputs.pack(side='top', fill='x', pady=20)
    search_button = ttk.Button(center_frame, text="Search", command=perform_search, style='TButton')
    search_button.pack(side='top', pady=20)

if __name__ == "__main__":
    setup_gui()
    root.mainloop()